#trivial Inner chunk

In [ ]:
# --- 0) Setup & imports ---
import glob, re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, ScalarFormatter

folder = "/content"

# If you want a metadata table:
try:
    import pandas as pd
except Exception:
    pd = None


# ---------------- add this near the top (config) ----------------
from pathlib import Path
save_figs   = True                 # <- turn saving on/off
out_dir     = Path(folder) / "topk_plots"
out_dir.mkdir(parents=True, exist_ok=True)
dpi_save    = 300                  # 300 dpi for print; bump if you like
# ----------------------------------------------------------------


# --- 1) Locate files in your Colab workspace ---
                # <-- change if your files are in a subfolder
pattern = f"{folder}/evolution_trivial_mag*.npz"
paths   = sorted(glob.glob(pattern))
assert paths, f"No NPZ files found at: {pattern}"

# --- 2) Helpers ---
def get_safe(d, key, default=None):
    return d[key] if key in d.files else default

def load_runs(paths):
    """
    Load saved runs into:
      results[mag] = {"metric": y, "evolution": (optional), "dt": dt, "timesteps": T, "g": g, "params": {...}}
    Also returns a metadata list for tabulation.
    """
    results = {}
    meta_rows = []
    for p in paths:
        data = np.load(p, allow_pickle=True)

        # Robust magnitude
        mag = get_safe(data, "mag")
        if mag is None:
            m = re.search(r"mag(\d+(\.\d+)?)", p)
            mag = float(m.group(1)) if m else None
        mag = float(mag)

        # Time & sim params
        dt  = float(get_safe(data, "dt", 1.0))
        T   = int(get_safe(data, "timesteps", 0)) or int(get_safe(data, "evolution").shape[0])
        g   = float(get_safe(data, "g", 10.0))  # fallback to 10 if not saved

        # Metric: prefer saved top6/top15; else compute from evolution
        if "top6_sums" in data.files:
            metric = data["top6_sums"]
        elif "top15_sums" in data.files:
            metric = data["top15_sums"]
        else:
            evo = data["evolution"]  # (T, N)
            metric = np.sort(evo, axis=1)[:, -6:].sum(axis=1)

        # (Optional) stash evolution if you want to recompute other things later
        evolution = get_safe(data, "evolution")

        # Collect other params if present
        params = {
            "n": get_safe(data, "n"),
            "m": get_safe(data, "m"),
            "c": get_safe(data, "c"),
            "cNN_mag": get_safe(data, "cNN_mag"),
            "phi": get_safe(data, "phi"),
            "M1": get_safe(data, "M1"),
            "M2": get_safe(data, "M2"),
            "chunk": get_safe(data, "chunk"),
            "init_indices": get_safe(data, "init_indices"),
            "nonlinear_sites": get_safe(data, "nonlinear_sites"),
            "created_utc": get_safe(data, "created_utc"),
        }

        results[mag] = {
            "metric": metric,
            "evolution": evolution,
            "dt": dt,
            "timesteps": len(metric),
            "g": g,
            "params": params,
            "path": p,
        }

        meta_rows.append({
            "file": p.split("/")[-1],
            "mag": mag,
            "g": g,
            "dt": dt,
            "timesteps": len(metric),
            "n": params["n"],
            "m": params["m"],
            "phi": params["phi"],
            "M1": params["M1"],
            "M2": params["M2"],
            "chunk": params["chunk"],
        })

    return results, meta_rows


results, meta_rows = load_runs(paths)
mag_values = sorted(results.keys())

# Optional: show a quick metadata table
if pd is not None:
    display(pd.DataFrame(meta_rows).sort_values("mag"))
else:
    for r in meta_rows: print(r)

# --- 3) Plotting helpers ---
def normalize_series(y, method="max"):
    y = np.asarray(y, float)
    if method == "max":
        scale = np.max(y)
    elif method == "t0":
        scale = y[0]
    elif method == "first_nonzero":
        nz = np.nonzero(y)[0]
        scale = y[nz[0]] if nz.size else 0.0
    else:
        raise ValueError("method must be 'max','t0','first_nonzero'")
    if scale <= 0 or not np.isfinite(scale):
        return np.zeros_like(y), 1.0
    return y / scale, scale

def half_life_index(y_norm):
    idx = np.where(y_norm <= 0.5)[0]
    return int(idx[0]) if idx.size else None

# --- 4) Non-normalized plot (absolute population) ---
fig, ax = plt.subplots(figsize=(16, 12), constrained_layout=True)
for mag in mag_values:
    y  = results[mag]["metric"] *10
    T  = results[mag]["timesteps"]
    x  = np.arange(T)  # or * results[mag]["dt"] for physical time
    g  = results[mag]["g"]
    nl_term = g * 5 * (mag**2)
    ax.plot(x, y, lw=2, label=fr"$|\psi_i(0)|={np.round(mag,2)},\; g\sum_{{j=1}}^5|\psi_j(0)|^2={nl_term}$")


ax.tick_params(
    axis="both",
    which="major",
    length=10,
    width=2,
    labelsize=20
)
ax.tick_params(
    axis="both",
    which="minor",
    length=6,
    width=1.5,
    labelsize=20
)


threshold_upp = 4
ax.axhline(threshold_upp, color='r', ls='--', lw=1.2, alpha=0.7)

threshold_bot = 6
ax.axhline(threshold_bot, color='r', ls='--', lw=1.2, alpha=0.7)


ax.set_xlabel("Time step", fontsize=20)
ax.set_ylabel("Edge-state population (top-5 sum)", fontsize=20)
# ax.set_title("Absolute Edge-State Population vs. Initial Amplitude", fontsize=16)
ax.xaxis.set_minor_locator(AutoMinorLocator()); ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.grid(which="major", linestyle="--", alpha=0.35); ax.grid(which="minor", linestyle=":", alpha=0.2)
for s in ("top","right"): ax.spines[s].set_visible(False)
ax.tick_params(axis="both", which="both", width=1.2); ax.tick_params(axis="both", which="major", length=6); ax.tick_params(axis="both", which="minor", length=3)
ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True)); ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
# ax.legend(frameon=False, fontsize=16)
# ax.set_xlim(0,6000)
ax.set_ylim(0,10)
plt.show()

if save_figs:
    fname_base = f"edge state multi non normalied trivial "
    fig.savefig(out_dir / f"{fname_base}.png", dpi=dpi_save, bbox_inches="tight")
    fig.savefig(out_dir / f"{fname_base}.pdf", dpi=dpi_save, bbox_inches="tight")
plt.show()
plt.close(fig)


# --- 5) Normalized plot (for decay comparison) ---
NORM_METHOD = "max"  # 'max' or 't0' or 'first_nonzero'
fig, ax = plt.subplots(figsize=(16, 12), constrained_layout=True)
half_lives = {}

for mag in mag_values:
    y  = results[mag]["metric"]
    T  = results[mag]["timesteps"]
    x  = np.arange(T)
    y_norm, _ = normalize_series(y, method=NORM_METHOD)
    g  = results[mag]["g"]
    nl_term = g * 5 * (mag**2)
    ax.plot(x, y_norm, lw=2, label=fr"$|\psi_i(0)|={np.round(mag,2)},\; g\sum_{{j=1}}^5|\psi_j(0)|^2={nl_term}$")

    hidx = half_life_index(y_norm)
    if hidx is not None:
        half_lives[mag] = {"idx": hidx, "time": hidx * results[mag]["dt"]}


ax.tick_params(
    axis="both",
    which="major",
    length=10,
    width=2,
    labelsize=18
)
ax.tick_params(
    axis="both",
    which="minor",
    length=6,
    width=1.5,
    labelsize=18
)


ax.set_xlabel("Time step", fontsize=20)
ax.set_ylabel("Normalized edge-state population", fontsize=20)
# ax.set_title("Normalized Evolution of Edge-State Population vs. Initial Amplitude", fontsize=16)
ax.xaxis.set_minor_locator(AutoMinorLocator()); ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.grid(which="major", linestyle="--", alpha=0.35); ax.grid(which="minor", linestyle=":", alpha=0.2)
for s in ("top","right"): ax.spines[s].set_visible(False)
ax.tick_params(axis="both", which="both", width=1.2); ax.tick_params(axis="both", which="major", length=6); ax.tick_params(axis="both", which="minor", length=3)
ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True)); ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
# ax.legend(frameon=False, fontsize=16)
plt.show()

if save_figs:
    fname_base = f"edge state multi normalied trivial "
    fig.savefig(out_dir / f"{fname_base}.png", dpi=dpi_save, bbox_inches="tight")
    fig.savefig(out_dir / f"{fname_base}.pdf", dpi=dpi_save, bbox_inches="tight")
plt.show()
plt.close(fig)


# Optional: print half-lives
for mag, hl in sorted(half_lives.items()):
    print(f"|psi_i(0)|={mag}: half-life ~ {hl['idx']} steps ({hl['time']:.3f} time units)")


In [ ]:
# === Load NPZ files and plot IPR without re-running dynamics ===
import glob, re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, ScalarFormatter

folder  = "/content"                       # change if needed
paths   = sorted(glob.glob(f"{folder}/evolution_trivial_mag*.npz"))
assert paths, "No NPZ files found."

def ipr_timeseries_from_npz(path):
    data = np.load(path, allow_pickle=True)
    # pull evolution (T, N) of |psi|^2 or |psi| (your files saved |psi|^2; if not, square it)
    evolution = data["evolution"]  # (T, N), assumed = |psi|^2 per your pipeline
    # IPR(t) = sum p^2 / (sum p)^2
    num = (evolution**2).sum(axis=1)
    den = (evolution.sum(axis=1))**2 + 1e-16
    ipr = num / den
    # metadata
    mag = float(data["mag"]) if "mag" in data.files else float(re.search(r"mag(\d+(\.\d+)?)", path).group(1))
    dt  = float(data["dt"]) if "dt" in data.files else 1.0
    g   = float(data["g"])  if "g"  in data.files else 10.0
    return ipr, mag, dt, g

# collect runs
runs = []
for p in paths:
    try:
        ipr, mag, dt, g = ipr_timeseries_from_npz(p)
        runs.append({"ipr": ipr, "mag": mag, "dt": dt, "g": g, "path": p})
    except Exception as e:
        print(f"Skipping {p}: {e}")

# sort by magnitude for tidy legends
runs = sorted(runs, key=lambda r: r["mag"])

# ---- plot: IPR vs time-step (or vs physical time) ----
fig, ax = plt.subplots(figsize=(16, 12), constrained_layout=True)

for r in runs:
    T = len(r["ipr"])
    x = np.arange(T)               # or: np.arange(T) * r["dt"] for physical time
    nl_term = r["g"] * 5 * (r["mag"]**2)
    ax.plot(x, r["ipr"], lw=2,
            label=fr"$|\psi_i(0)|={np.round(r['mag'],2)},\; g\sum_{{j=1}}^5|\psi_j(0)|^2={nl_term}$")

ax.set_xlabel("Time step", fontsize=20 )
ax.set_ylabel("IPR (dimensionless)", fontsize=20)
# ax.set_title("IPR vs. time (loaded from saved evolutions)")

threshold_upp = 0.004
ax.axhline(threshold_upp, color='r', ls='--', lw=1.2, alpha=0.7)

threshold_bot = 0.0038
ax.axhline(threshold_bot, color='r', ls='--', lw=1.2, alpha=0.7)

ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.grid(which="major", linestyle="--", alpha=0.35)
ax.grid(which="minor", linestyle=":", alpha=0.2)

for spine in ("top", "right"):
    ax.spines[spine].set_visible(False)
ax.tick_params(axis="both", which="both", width=1.2)
ax.tick_params(axis="both", which="major", length=6)
ax.tick_params(axis="both", which="minor", length=3)

ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
# ax.set_xlim(0,6000)
ax.set_ylim(0,0.01)

ax.tick_params(
    axis="both",
    which="major",
    length=10,    # ← longer major ticks
    width=2,      # ← thicker tick lines
    labelsize=24  # ← bigger tick label font
)
ax.tick_params(
    axis="both",
    which="minor",
    length=6,     # ← longer minor ticks
    width=1.5,    # ← thicker minor tick lines
    labelsize=20
)



# ax.legend(frameon=False, fontsize=16)
plt.show()

if save_figs:
    fname_base = f"ipr trivial  "
    fig.savefig(out_dir / f"{fname_base}.png", dpi=dpi_save, bbox_inches="tight")
    fig.savefig(out_dir / f"{fname_base}.pdf", dpi=dpi_save, bbox_inches="tight")
plt.show()
plt.close(fig)


In [ ]:
# --- Animate a saved run from results[mag]["evolution"] ---
def generate_zigzag_coords(n, m):
    """
    Returns X, Y coordinate arrays for zigzag nanoribbon sites (interleaved A and B).
    A-sites at even indices, B-sites at odd indices.
    """
    # Lattice vectors
    a1 = np.array([1.5,  np.sqrt(3)/2])
    a2 = np.array([1.5, -np.sqrt(3)/2])
    delta = np.array([[0, 0], [1, 0]])  # A and B sublattice offsets

    # Generate grid of unit cells
    IX, IY = np.meshgrid(np.arange(n), np.arange(m))
    IX = IX.flatten()
    IY = IY.flatten()
    N_uc = len(IX)
    N = 2 * N_uc

    # Allocate coordinate arrays
    X = np.zeros(N)
    Y = np.zeros(N)

    # A sublattice positions
    A_x = IX * a1[0] + IY * a2[0] + delta[0, 0]
    A_y = IX * a1[1] + IY * a2[1] + delta[0, 1]

    # B sublattice positions
    B_x = IX * a1[0] + IY * a2[0] + delta[1, 0]
    B_y = IX * a1[1] + IY * a2[1] + delta[1, 1]

    # Interleave A and B into single arrays
    X[0::2] = A_x
    Y[0::2] = A_y
    X[1::2] = B_x
    Y[1::2] = B_y

    return X, Y

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML

mag = 1  # <- choose which run to animate
evolution = results[mag]["evolution"]               # shape: (T, N)
T = evolution.shape[0]
X, Y = generate_zigzag_coords(int(results[mag]["params"]["n"]), int(results[mag]["params"]["m"]))  # or use your globals n,m

# Nice color scaling from data (ignore extreme outliers)
vmin = np.percentile(evolution, 1.0)
vmax = 0.1 #np.percentile(evolution, 99.0)

plt.rcParams['animation.embed_limit'] = 500  # MB

fig, ax = plt.subplots(figsize=(10, 10))

scat = ax.scatter(X, Y, c=evolution[0], cmap='viridis', s=90, vmin=vmin, vmax=vmax)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%", pad=0.1)
cbar = plt.colorbar(scat, cax=cax)
cbar.set_label('Probability Density', fontsize=16)
cbar.ax.tick_params(labelsize=14)

ax.set_xlabel("x (a.u.)", fontsize=18)
ax.set_ylabel("y (a.u.)", fontsize=18)
ax.tick_params(axis='both', labelsize=16)
ax.set_aspect('equal', adjustable='box')

# Choose which frames to show (e.g., every 200th)
frame_step = 100
frames = range(0, T, frame_step)

def update(frame_idx):
    scat.set_array(evolution[frame_idx])
    ax.set_title(f"Time Step: {frame_idx}", fontsize=20)
    return scat,

ani = FuncAnimation(fig, update, frames=frames, blit=True)
plt.close(fig)

HTML(ani.to_jshtml())


In [ ]:
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

def save_animation_frames(results, mag, step=1000, out_root="frames",
                          dpi=150, fmt="png", cmap="viridis",
                          vmin=0, vmax=mag, percentiles=(1, 99),
                          figsize=(10, 10), marker_size=90):
    """
    Save scatter frames of results[mag]["evolution"] every `step` timesteps.

    - results: dict holding "evolution" (T, N) and params n,m
    - mag: which run to export (folder name will be out_root/mag_{mag})
    - step: frame stride, e.g. 1000 to save t=0,1000,2000,...
    - vmin/vmax: color scale; if None, computed from percentiles over all data
    - fmt: image format ("png", "jpg", etc.)
    """
    evolution = results[mag]["evolution"]          # shape (T, N)
    T, N = evolution.shape
    n = int(results[mag]["params"]["n"])
    m = int(results[mag]["params"]["m"])

    # your lattice coords
    X, Y = generate_zigzag_coords(n, m)

    # output dir
    out_dir = Path(out_root) / f"mag_{mag}"
    out_dir.mkdir(parents=True, exist_ok=True)

    # color scale (fixed across frames)
    if vmin is None or vmax is None:
        p_lo, p_hi = percentiles
        if vmin is None:
            vmin = np.percentile(evolution, p_lo)
        if vmax is None:
            vmax = np.percentile(evolution, p_hi)

    fig, ax = plt.subplots(figsize=figsize)
    scat = ax.scatter(X, Y, c=evolution[0], cmap=cmap, s=marker_size,
                      vmin=vmin, vmax=0.1)

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.1)
    cbar = plt.colorbar(scat, cax=cax)
    cbar.set_label('Probability Density', fontsize=16)
    cbar.ax.tick_params(labelsize=14)

    ax.set_xlabel("x (a.u.)", fontsize=22)
    ax.set_ylabel("y (a.u.)", fontsize=22)
    ax.tick_params(axis='both', labelsize=22)
    ax.set_aspect('equal', adjustable='box')

    # loop over frames and save
    count = 0
    for t in range(0, T, step):
        scat.set_array(evolution[t])
        ax.set_title(f"Time Step: {t}", fontsize=20)
        fig.savefig(out_dir / f"frame_{t:06d}_mag={mag}.{fmt}",
                    dpi=dpi, bbox_inches="tight")
        count += 1

    plt.close(fig)
    print(f"Saved {count} frames to: {out_dir}")

# --- usage ---
# mag = 6
save_animation_frames(results, mag, step=100, out_root=f"frames_mag={mag}",
                      dpi=150, fmt="png", percentiles=(1, 99))


Saved 200 frames to: frames_mag=1/mag_1


# non trivial chunk

In [ ]:
# === Load NPZ files and plot Top-6 sums (no re-run) ===
import glob, re, os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, ScalarFormatter

# ---------------- config ----------------
folder   = "/content"                               # change if needed
pattern  = f"{folder}/evolution_non_trivial_mag*.npz"   # or "evolution_mag*.npz"
k_top    = 6                                        # top-k sites per frame
use_time = "steps"                                  # "steps" or "physical"
plot_normalized = True                              # also show normalized curves

# ---------------- add this near the top (config) ----------------
from pathlib import Path
save_figs   = True                 # <- turn saving on/off
out_dir     = Path(folder) / "topk_plots"
out_dir.mkdir(parents=True, exist_ok=True)
dpi_save    = 300                  # 300 dpi for print; bump if you like
# ----------------------------------------------------------------


# ------------- helpers -------------
def normalize_series(y, method="max"):
    y = np.asarray(y, float)
    scale = np.max(y) if method == "max" else y[0]
    if scale <= 0 or not np.isfinite(scale):
        return np.zeros_like(y), 1.0
    return y/scale, scale

def load_topk_sum_from_npz(path, k=6):
    """
    Load evolution from NPZ and compute top-k sum per frame.
    Returns: topk_sum(t), mag, dt, g, T
    """
    data = np.load(path, allow_pickle=True)
    if "evolution" not in data.files:
        raise ValueError(f"{os.path.basename(path)} missing 'evolution'")
    evolution = np.asarray(data["evolution"], dtype=float)  # (T, N), stores |psi|^2
    T = evolution.shape[0]

    # compute top-k sum per frame efficiently (no full sort)
    part = np.partition(evolution, -k, axis=1)[:, -k:]      # shape (T, k)
    topk_sums = part.sum(axis=1)                            # (T,)

    # metadata for labels / x-axis
    mag = float(data["mag"]) if "mag" in data.files else float(re.search(r"mag(\d+(\.\d+)?)", path).group(1))
    dt  = float(data["dt"]) if "dt" in data.files else 1.0
    g   = float(data["g"])  if "g"  in data.files else 10.0

    return topk_sums, mag, dt, g, T

def x_axis(T, dt, mode):
    return np.arange(T) * dt if mode == "physical" else np.arange(T)

# ------------- load all runs -------------
paths = sorted(glob.glob(pattern))
assert paths, f"No NPZ files found for pattern: {pattern}"

runs = []
for p in paths:
    try:
        y, mag, dt, g, T = load_topk_sum_from_npz(p, k=k_top)
        runs.append({"y": y, "mag": mag, "dt": dt, "g": g, "T": T, "path": p})
    except Exception as e:
        print(f"Skipping {p}: {e}")

# tidy legend order
runs = sorted(runs, key=lambda r: r["mag"])

# ------------- PLOT: absolute (non-normalized) -------------
fig, ax = plt.subplots(figsize=(16, 12), constrained_layout=True)

for r in runs:
    x = x_axis(r["T"], r["dt"], use_time)
    nl_term = int(r["g"] * 5 * (r["mag"]**2))
    ax.plot(x, r["y"]*10, lw=2,
            label=rf"$|\psi_i(0)|={round(r['mag'],2)},\; g\sum_{{j=1}}^{5}|\psi_j(0)|^2={nl_term}$")

ax.set_xlabel("Time [steps]" if use_time=="steps" else "Time", fontsize=24)
ax.set_ylabel(f"Top-{k_top} population sum", fontsize=24)
# ax.set_title(f"Edge-State Population (Top-{k_top} Sum) from saved evolutions", fontsize=16)

# Increase tick label size and tick length/width
ax.tick_params(axis="both", which="major", labelsize=20, length=8, width=2)
ax.tick_params(axis="both", which="minor", labelsize=16, length=5, width=1.5)


# threshold_upp = 4
# ax.axhline(threshold_upp, color='r', ls='--', lw=1.2, alpha=0.7)

# threshold_bot = 6
# ax.axhline(threshold_bot, color='r', ls='--', lw=1.2, alpha=0.7)


ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.grid(which="major", linestyle="--", alpha=0.35)
ax.grid(which="minor", linestyle=":", alpha=0.2)
for spine in ("top", "right"):
    ax.spines[spine].set_visible(False)
ax.tick_params(axis="both", which="both", width=1.2)
ax.tick_params(axis="both", which="major", length=6)
ax.tick_params(axis="both", which="minor", length=3)
ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax.legend(frameon=False, fontsize=16)
ax.set_ylim(0,10)
plt.show()

# ------------- PLOT: absolute (non-normalized) -------------


if save_figs:
    fname_base = f"edge_top{k_top}_sum_{use_time}"
    fig.savefig(out_dir / f"{fname_base}.png", dpi=dpi_save, bbox_inches="tight")
    fig.savefig(out_dir / f"{fname_base}.pdf", dpi=dpi_save, bbox_inches="tight")
plt.show()
if save_figs:
        fname_base = f"edge_top6_sum_steps-NON_Normalized-NONTRIVIAL_CHUINK_zoomed"
        fig.savefig(out_dir / f"{fname_base}.png", dpi=dpi_save, bbox_inches="tight")
        fig.savefig(out_dir / f"{fname_base}.pdf", dpi=dpi_save, bbox_inches="tight")
plt.close(fig)



# ------------- PLOT: normalized (per-run) -------------
if plot_normalized:
    fig, ax = plt.subplots(figsize=(10, 6), constrained_layout=True)
    for r in runs:
        y_norm, _ = normalize_series(r["y"], method="max")  # or method="t0"
        x = x_axis(r["T"], r["dt"], use_time)
        ax.plot(x, y_norm, lw=2, label=rf"$|\psi_i(0)|={r['mag']}$")

    ax.set_xlabel("Time [steps]" if use_time=="steps" else "Time", fontsize=20)
    ax.set_ylabel(f"Normalized top-{k_top} sum", fontsize=20)
    # ax.set_title(f"Normalized Edge-State Population (Top-{k_top})", fontsize=20)

    ax.axhline(0.5, color='k', ls='--', lw=1.2, alpha=0.7)
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.grid(which="major", linestyle="--", alpha=0.35)
    ax.grid(which="minor", linestyle=":", alpha=0.2)
    for spine in ("top", "right"):
        ax.spines[spine].set_visible(False)
    ax.tick_params(axis="both", which="both", width=1.2,labelsize=20)
    ax.tick_params(axis="both", which="major", length=6, labelsize=16)
    ax.tick_params(axis="both", which="minor", length=3)
    ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    ax.legend(frameon=False, fontsize=11)
    plt.show()



    # if save_figs:
    #     fname_base = f"edge_top6_sum_steps-NON_Normalized-NONTRIVIAL_CHUINK_zoomed"
    #     fig.savefig(out_dir / f"{fname_base}.png", dpi=dpi_save, bbox_inches="tight")
    #     fig.savefig(out_dir / f"{fname_base}.pdf", dpi=dpi_save, bbox_inches="tight")
    # plt.show()
    # plt.close(fig)



In [ ]:
# === Load NPZ files and plot IPR without re-running dynamics ===
import glob, re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, ScalarFormatter

folder  = "/content"                       # change if needed
paths   = sorted(glob.glob(f"{folder}/evolution_non_trivial_mag*.npz"))
assert paths, "No NPZ files found."

def ipr_timeseries_from_npz(path):
    data = np.load(path, allow_pickle=True)
    # pull evolution (T, N) of |psi|^2 or |psi| (your files saved |psi|^2; if not, square it)
    evolution_non_trivial = data["evolution"]  # (T, N), assumed = |psi|^2 per your pipeline
    # IPR(t) = sum p^2 / (sum p)^2
    num = (evolution_non_trivial**2).sum(axis=1)
    den = (evolution_non_trivial.sum(axis=1))**2 + 1e-16
    ipr = num / den
    # metadata
    mag = float(data["mag"]) if "mag" in data.files else float(re.search(r"mag(\d+(\.\d+)?)", path).group(1))
    dt  = float(data["dt"]) if "dt" in data.files else 1.0
    g   = float(data["g"])  if "g"  in data.files else 10.0
    return ipr, mag, dt, g

# collect runs
runs = []
for p in paths:
    try:
        ipr, mag, dt, g = ipr_timeseries_from_npz(p)
        runs.append({"ipr": ipr, "mag": mag, "dt": dt, "g": g, "path": p})
    except Exception as e:
        print(f"Skipping {p}: {e}")

# sort by magnitude for tidy legends
runs = sorted(runs, key=lambda r: r["mag"])

# ---- plot: IPR vs time-step (or vs physical time) ----
fig, ax = plt.subplots(figsize=(16, 12), constrained_layout=True)

for r in runs:
    T = len(r["ipr"])
    x = np.arange(T)               # or: np.arange(T) * r["dt"] for physical time
    nl_term = r["g"] * 5 * (r["mag"]**2)
    ax.plot(x, r["ipr"], lw=2,
            label=fr"$|\psi_i(0)|={np.round(r['mag'],2)},\; g\sum_{{j=1}}^5|\psi_j(0)|^2={int(nl_term)}$")


threshold_upp = 0.004
ax.axhline(threshold_upp, color='r', ls='--', lw=1.2, alpha=0.7)

threshold_bot = 0.0038
ax.axhline(threshold_bot, color='r', ls='--', lw=1.2, alpha=0.7)


ax.set_xlabel("Time step", fontsize=24)
ax.set_ylabel("IPR (dimensionless)", fontsize=24)
# ax.set_title("IPR vs. time (loaded from saved evolutions)")
# ax.set_ylim(0.0, 0.01)
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.yaxis.set_minor_locator(AutoMinorLocator())
ax.grid(which="major", linestyle="--", alpha=0.35)
ax.grid(which="minor", linestyle=":", alpha=0.2)

for spine in ("top", "right"):
    ax.spines[spine].set_visible(False)
ax.tick_params(axis="both", which="both", width=1.2,labelsize= 20)
ax.tick_params(axis="both", which="major", length=6,labelsize= 20)
ax.tick_params(axis="both", which="minor", length=3,labelsize= 20)

ax.yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=True))

ax.legend(frameon=False, fontsize=16)
# ax.set_ylim(0,0.01)
plt.show()

if save_figs:
  fname_base = f"IPR_multi_nontrivial_chunk"
  fig.savefig(out_dir / f"{fname_base}.png", dpi=dpi_save, bbox_inches="tight")
  # fig.savefig(out_dir / f"{fname_base}.pdf", dpi=dpi_save, bbox_inches="tight")
plt.show()
plt.close(fig)


In [ ]:
# === Animate a saved run from a selected NPZ file (no re-run) ===
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML

# ---------- pick the file you want to animate ----------
file_path = "/content/evolution_non_trivial_mag1 (2).npz"   # <- change to your file

# ---------- zigzag coords (same layout you used) ----------
def generate_zigzag_coords(n, m):
    """
    Returns X, Y coordinate arrays for zigzag nanoribbon sites (interleaved A and B).
    A-sites at even indices, B-sites at odd indices.
    """
    a1 = np.array([1.5,  np.sqrt(3)/2])
    a2 = np.array([1.5, -np.sqrt(3)/2])
    delta = np.array([[0, 0], [1, 0]])  # A and B sublattice offsets

    IX, IY = np.meshgrid(np.arange(n), np.arange(m))
    IX = IX.flatten()
    IY = IY.flatten()
    N_uc = len(IX)
    N = 2 * N_uc

    X = np.zeros(N)
    Y = np.zeros(N)

    A_x = IX * a1[0] + IY * a2[0] + delta[0, 0]
    A_y = IX * a1[1] + IY * a2[1] + delta[0, 1]
    B_x = IX * a1[0] + IY * a2[0] + delta[1, 0]
    B_y = IX * a1[1] + IY * a2[1] + delta[1, 1]

    X[0::2] = A_x; Y[0::2] = A_y
    X[1::2] = B_x; Y[1::2] = B_y
    return X, Y

# ---------- load saved evolution ----------
data = np.load(file_path, allow_pickle=True)
evolution = np.asarray(data["evolution"], dtype=float)    # shape (T, N), stores |psi|^2
T, N = evolution.shape

# lattice sizes (prefer from file)
if "n" in data.files and "m" in data.files:
    n = int(data["n"]); m = int(data["m"])
else:
    # fallback — set manually if your NPZ lacks n,m
    n, m = 20, 20

dt = float(data["dt"]) if "dt" in data.files else 1.0
mag = float(data["mag"]) if "mag" in data.files else None

X, Y = generate_zigzag_coords(n, m)
assert len(X) == N, f"Coordinate count {len(X)} != N {N}"

# color scale (robust to outliers)
vmin = np.nanpercentile(evolution, 1.0)
vmax = 0.1 # np.nanpercentile(evolution, 99.0)
# if you prefer a fixed cap (e.g., 1.0), set: vmax = 1.0

plt.rcParams['animation.embed_limit'] = 500  # MB limit for notebook embedding

fig, ax = plt.subplots(figsize=(10, 9))
scat = ax.scatter(X, Y, c=evolution[0], cmap='viridis', s=90, vmin=vmin, vmax=vmax)

# colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%", pad=0.1)
cbar = plt.colorbar(scat, cax=cax)
cbar.set_label('Probability Density', fontsize=14)
cbar.ax.tick_params(labelsize=12)

ax.set_xlabel("x (a.u.)", fontsize=14)
ax.set_ylabel("y (a.u.)", fontsize=14)
ax.tick_params(axis='both', labelsize=12)
ax.set_aspect('equal', adjustable='box')
ax.set_title("Time Step: 0", fontsize=16)

# which frames to show
frame_step = 100                    # e.g., every 50th frame
frames = range(0, T, frame_step)  # or a list of specific indices

def update(frame_idx):
    scat.set_array(evolution[frame_idx])
    t_phys = frame_idx * dt
    if mag is not None:
        ax.set_title(f"Time Step: {frame_idx}   (t = {t_phys:.3f})   |ψ_i(0)| = {mag}", fontsize=16)
    else:
        ax.set_title(f"Time Step: {frame_idx}   (t = {t_phys:.3f})", fontsize=16)
    return scat,

ani = FuncAnimation(fig, update, frames=frames, blit=True, interval=80)
plt.close(fig)

# show in notebook
HTML(ani.to_jshtml())

# ---- Optional: save to file (needs ffmpeg for mp4) ----
# ani.save("evolution.mp4", fps=12, dpi=150)
# ani.save("evolution.gif", fps=12, dpi=100)


In [ ]:
# --- Save frames (same style as your animation) from a saved NPZ run ---
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pathlib import Path

# ---------- zigzag coords (same as in your animation) ----------
def generate_zigzag_coords(n, m):
    a1 = np.array([1.5,  np.sqrt(3)/2])
    a2 = np.array([1.5, -np.sqrt(3)/2])
    delta = np.array([[0, 0], [1, 0]])  # A and B sublattice offsets
    IX, IY = np.meshgrid(np.arange(n), np.arange(m))
    IX = IX.flatten(); IY = IY.flatten()
    N_uc = len(IX); N = 2 * N_uc
    X = np.zeros(N); Y = np.zeros(N)
    A_x = IX * a1[0] + IY * a2[0] + delta[0, 0]
    A_y = IX * a1[1] + IY * a2[1] + delta[0, 1]
    B_x = IX * a1[0] + IY * a2[0] + delta[1, 0]
    B_y = IX * a1[1] + IY * a2[1] + delta[1, 1]
    X[0::2] = A_x; Y[0::2] = A_y
    X[1::2] = B_x; Y[1::2] = B_y
    return X, Y

def save_frames_same_style(file_path, frame_step=1000, out_root="frames", fmt="png", dpi=150):
    # --- load saved evolution ---
    data = np.load(file_path, allow_pickle=True)
    evolution = np.asarray(data["evolution"], dtype=float)   # (T, N) with |psi|^2
    T, N = evolution.shape

    # lattice sizes
    n = int(data["n"]) if "n" in data.files else 20
    m = int(data["m"]) if "m" in data.files else 20
    dt = float(data["dt"]) if "dt" in data.files else 1.0
    mag = float(data["mag"]) if "mag" in data.files else None

    X, Y = generate_zigzag_coords(n, m)
    assert len(X) == N, f"Coordinate count {len(X)} != N {N}"

    # --- same color scaling as animation (robust percentiles) ---
    vmin = np.nanpercentile(evolution, 1.0)
    vmax = 0.1 #np.nanpercentile(evolution, 99.0)
    # (If you prefer a fixed cap, e.g. vmax = 1.0, set it here.)

    # --- output folder named from file (and mag if present) ---
    stem = Path(file_path).stem
    out_dir = Path(out_root) / (f"{stem}_mag_{int(mag) if isinstance(mag, float) and mag.is_integer() else mag}"
                                if mag is not None else stem)
    out_dir.mkdir(parents=True, exist_ok=True)

    # --- figure with identical style to your animation ---
    fig, ax = plt.subplots(figsize=(10, 9))  # same figsize
    scat = ax.scatter(X, Y, c=evolution[0], cmap='viridis', s=90, vmin=vmin, vmax=vmax)  # same s=90, cmap

    # colorbar with same font sizes
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.1)
    cbar = plt.colorbar(scat, cax=cax)
    cbar.set_label('Probability Density', fontsize=18)
    cbar.ax.tick_params(labelsize=12)

    # axes + title font sizes (match your animation)
    ax.set_xlabel("x (a.u.)", fontsize=22)
    ax.set_ylabel("y (a.u.)", fontsize=22)
    ax.tick_params(axis='both', labelsize=22)
    ax.set_aspect('equal', adjustable='box')

    # --- save frames at the same sampled indices you animated ---
    frames = range(0, T, frame_step)
    count = 0
    for t in frames:
        scat.set_array(evolution[t])
        t_phys = t * dt
        if mag is not None:
            ax.set_title(f"Time Step: {t}   (t = {t_phys:.3f})   |ψ_i(0)| = {mag}", fontsize=16)
        else:
            ax.set_title(f"Time Step: {t}   (t = {t_phys:.3f})", fontsize=16)

        out_file = out_dir / f"frame_{t:06d}.{fmt}"
        fig.savefig(out_file, dpi=dpi, bbox_inches="tight")
        count += 1

    plt.close(fig)
    print(f"Saved {count} frames to: {out_dir}")

# --- usage ---
file_path = "/content/evolution_non_trivial_mag1 (2).npz"
save_frames_same_style(file_path, frame_step=100, out_root="frames", fmt="png", dpi=150)


Saved 200 frames to: frames/evolution_non_trivial_mag1 (2)_mag_1
